<a href="https://colab.research.google.com/github/ephrem823/ML/blob/main/new_one.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import re
from collections import Counter

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense, Dropout


In [ ]:
with open('AmQA_Dataset.json', 'r', encoding='utf-8') as f:
    data_loaded = json.load(f)

qas = []
for art in data_loaded["data"]:
    paragraphs_raw = art["paragraphs"]
    if isinstance(paragraphs_raw, dict):
        paragraphs_to_process = [paragraphs_raw]
    elif isinstance(paragraphs_raw, list):
        paragraphs_to_process = paragraphs_raw
    else:
        continue

    for paragraph_data in paragraphs_to_process:
        context = paragraph_data["context"]
        for q in paragraph_data["qas"]:
            qas.append({
                "context": context,
                "question": q["question"],
                "answer": q["answers"][0]["text"]
            })

df_qas = pd.DataFrame(qas)
display(df_qas.head(2))

,context,question,answer
0,ንጉሡ ላሊበላ የሚለውን ስም ያገኘው፣ ሲወለድ በንቦች ስለተከበበ ነው ይባ...,በላሊበላ ስንት ውቅር አብያተ ክርስቲያናት አሉ?,11
1,ንጉሡ ላሊበላ የሚለውን ስም ያገኘው፣ ሲወለድ በንቦች ስለተከበበ ነው ይባ...,ከላሊበላ አስራ አንዱ ውቅር አብያተ ክርስቲያናት ግዙፉ የትኛው ነው?,ቤተ መድሃኔ ዓለም


In [ ]:
def clean(text):
    if not isinstance(text, str):
        return ""
    text = re.sub(r"[0-9A-Za-z!?,;:()\"'’\-\.]", " ", text)
    text = re.sub(r"\s+", " ", text)
    return text.strip()

df_qas["cleanedquestion"] = df_qas["question"].apply(clean)
df_qas["cleanedanswer"] = df_qas["answer"].apply(clean)

df_qas[["question", "cleanedquestion", "answer", "cleanedanswer"]].head(3)


,question,cleanedquestion,answer,cleanedanswer
0,በላሊበላ ስንት ውቅር አብያተ ክርስቲያናት አሉ?,በላሊበላ ስንት ውቅር አብያተ ክርስቲያናት አሉ,11,
1,ከላሊበላ አስራ አንዱ ውቅር አብያተ ክርስቲያናት ግዙፉ የትኛው ነው?,ከላሊበላ አስራ አንዱ ውቅር አብያተ ክርስቲያናት ግዙፉ የትኛው ነው,ቤተ መድሃኔ ዓለም,ቤተ መድሃኔ ዓለም
2,ከላሊበላ አስራ አንዱ ውቅር አብያተ ክርስቲያናት ቤተ ጊዮርጊስ ምን ዓይነ...,ከላሊበላ አስራ አንዱ ውቅር አብያተ ክርስቲያናት ቤተ ጊዮርጊስ ምን ዓይነ...,መስቀል,መስቀል


In [ ]:

ans_counts = Counter(df_qas["cleanedanswer"])
print("Total unique cleaned answers:", len(ans_counts))

top_k = 50
top_answers = set([a for a, c in ans_counts.most_common(top_k)])

df_qas["answer_reduced"] = df_qas["cleanedanswer"].apply(
    lambda a: a if a in top_answers else "<OTHER>"
)

print("Unique reduced answers:", df_qas["answer_reduced"].nunique())
df_qas["answer_reduced"].value_counts().head(10)


Total unique cleaned answers: 2050
Unique reduced answers: 51


,count
answer_reduced,
<OTHER>,2174
,75
በ,51
በ ዓ ም,35
በ እ ኤ አ,33
%,13
በ ዓክልበ,11
ሁለት,11
በ ዓም,10


In [ ]:
texts = df_qas["cleanedquestion"].astype(str).tolist()
labels = df_qas["answer_reduced"].astype(str).tolist()

vocab_size = 10000
max_length = 40

tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(texts)

sequences = tokenizer.texts_to_sequences(texts)
X = pad_sequences(sequences, maxlen=max_length, padding="post", truncating="post")

print("X shape:", X.shape)

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(labels)
num_classes = len(label_encoder.classes_)

print("Number of classes (reduced):", num_classes)
print("Example classes:", list(label_encoder.classes_)[:10])
print("y shape:", y.shape)


X shape: (2622, 40)
Number of classes (reduced): 51
Example classes: [np.str_(''), np.str_('%'), np.str_('<OTHER>'), np.str_('ሁለተኛ'), np.str_('ሁለት'), np.str_('ለ'), np.str_('ለ ዓመታት'), np.str_('ሐምሌ ፲፮ ቀን ፲፰፻፹፬ ዓ ም'), np.str_('ሚሊዮን'), np.str_('ሜትር')]
y shape: (2622,)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print("Train X:", X_train.shape)
print("Test X:", X_test.shape)
print("Train y:", y_train.shape)
print("Test y:", y_test.shape)

unique, counts = np.unique(y_train, return_counts=True)
print("Train class counts (id:count) for first 10 classes:",
      dict(list(zip(unique, counts))[:10]))


Train X: (2097, 40)
Test X: (525, 40)
Train y: (2097,)
Test y: (525,)
Train class counts (id:count) for first 10 classes: {np.int64(0): np.int64(60), np.int64(1): np.int64(10), np.int64(2): np.int64(1739), np.int64(3): np.int64(3), np.int64(4): np.int64(9), np.int64(5): np.int64(4), np.int64(6): np.int64(3), np.int64(7): np.int64(3), np.int64(8): np.int64(5), np.int64(9): np.int64(6)}


In [ ]:
tf.random.set_seed(42)

model = Sequential([
    Embedding(input_dim=vocab_size,
              output_dim=64,
              input_length=max_length),
    GlobalAveragePooling1D(),
    Dense(64, activation="relu"),
    Dropout(0.4),
    Dense(num_classes, activation="softmax")
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ ?                      │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
batch_size = 32
epochs = 30

history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=epochs,
    batch_size=batch_size,
    verbose=1
)

Epoch 1/30
66/66 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.7206 - loss: 2.7741 - val_accuracy: 0.8286 - val_loss: 1.1129
Epoch 2/30
66/66 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.8282 - loss: 1.1867 - val_accuracy: 0.8286 - val_loss: 1.0676
Epoch 3/30
66/66 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.8282 - loss: 1.1582 - val_accuracy: 0.8286 - val_loss: 1.0585
Epoch 4/30
66/66 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.8282 - loss: 1.1212 - val_accuracy: 0.8286 - val_loss: 1.0567
Epoch 5/30
66/66 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.8282 - loss: 1.1048 - val_accuracy: 0.8286 - val_loss: 1.0496
Epoch 6/30
66/66 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.8282 - loss: 1.1187 - val_accuracy: 0.8286 - val_loss: 1.0475
Epoch 7/30
66/66 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.8282 - loss: 1.0837 - val_accuracy: 0.8286 - val_loss: 1.0416
Epoch 8/30
66/66 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.8282 - loss: 1.0697 - val_accuracy: 0.8286 - val_l

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(f"Test accuracy: {test_acc:.4f}")
print(f"Test loss: {test_loss:.4f}")

Test accuracy: 0.8229
Test loss: 1.1882


In [ ]:
def predict_answer_nn(question_text: str):
    """
    Predict answer using the trained neural model.
    Returns: (predicted_answer, confidence_score)
    """
    cleaned_q = clean(question_text)

    seq = tokenizer.texts_to_sequences([cleaned_q])
    padded = pad_sequences(seq, maxlen=max_length, padding="post", truncating="post")

    probs = model.predict(padded, verbose=0)

    pred_class = np.argmax(probs, axis=1)[0]
    pred_answer = label_encoder.inverse_transform([pred_class])[0]
    confidence = float(np.max(probs))

    return pred_answer, confidence

print("=" * 60)
print("AMHARIC QA SYSTEM - POWERED BY NEURAL NETWORK")
print("=" * 60)

while True:
    print("\nEnter your question (or type 'quit' to exit):")
    user_question = input("> ").strip()

    if user_question.lower() in ["quit", "exit", "q"]:
        print("Thank you for using the QA system. Goodbye!")
        break

    if not user_question:
        print("Please enter a valid question.")
        continue

    predicted_answer, confidence_score = predict_answer_nn(user_question)

    # Display results
    print("\n" + "-" * 60)
    print(f"📝 Your Question: {user_question}")
    print(f"✅ Predicted Answer: {predicted_answer}")
    print(f"📊 Confidence Score: {confidence_score:.4f} ({confidence_score*100:.2f}%)")
    print("-" * 60)


    similar_idx = None
    max_sim = 0

    cleaned_user = clean(user_question)
    user_tokens = set(cleaned_user.split())

    for idx, row in df_qas.iterrows():
        cleaned_data = clean(row["cleanedquestion"])
        data_tokens = set(cleaned_data.split())


        if len(user_tokens | data_tokens) > 0:
            similarity = len(user_tokens & data_tokens) / len(user_tokens | data_tokens)
            if similarity > max_sim:
                max_sim = similarity
                similar_idx = idx

    if similar_idx is not None and max_sim > 0.3:
        print(f"\n🔍 Similar question in dataset (similarity: {max_sim:.2f}):")
        print(f"   Q: {df_qas.iloc[similar_idx]['question']}")
        print(f"   A: {df_qas.iloc[similar_idx]['answer']}")

    print()

AMHARIC QA SYSTEM - POWERED BY NEURAL NETWORK

Enter your question (or type 'quit' to exit):

------------------------------------------------------------
📝 Your Question: ከላሊበላ አስራ አንዱ ውቅር አብያተ ክርስቲያናት ግዙፉ የትኛው ነው
✅ Predicted Answer: <OTHER>
📊 Confidence Score: 0.9980 (99.80%)
------------------------------------------------------------

🔍 Similar question in dataset (similarity: 1.00):
   Q: ከላሊበላ አስራ አንዱ ውቅር አብያተ ክርስቲያናት ግዙፉ የትኛው ነው?
   A: ቤተ መድሃኔ ዓለም


Enter your question (or type 'quit' to exit):
